## Create dataframes for visualization

In [29]:
import nibabel as nib
from pathlib import Path
from tqdm import tqdm
from matplotlib import pyplot as plt
from parse import parse
import numpy as np
import os 
import pandas as pd 
from hedypet.utils import get_norm_consts, get_participant_metadata
from hedypet.utils import load_splits 

def filter_and_combine_regions(df,regions):
    group_cols = ["Subject","Erosion Iterations","Task"]
    if "Frame Index" in df:
        group_cols+=["Frame Index"]
    results = []

    for k, v in regions.items():
        mask = df["Label Name"].isin(v) if isinstance(v, list) else df["Label Name"] == v
        filtered = df[mask]
        
        if isinstance(v, list):
            result = filtered.groupby(group_cols).apply(
                lambda x: pd.Series({
                    'mu': (x["PET Mean [Bq/mL]"] * x["Voxel Count"]).sum() / x["Voxel Count"].sum(),
                    "Volume [mL]": x["Volume [mL]"].sum()
                })
            ).reset_index()
        else:
            result = filtered
            result["mu"] = result["PET Mean [Bq/mL]"]
            result["Volume [mL]"] = result["Volume [mL]"]
        
        result['region'] = k
        results.append(result)
    df = pd.concat(results, ignore_index=True)
    return df.drop("Label Index",axis=1)

regions = {
    "Spleen":"spleen",
    "Kidneys": ["kidney_right","kidney_left"],
    "Aorta": "aorta",
    "Liver":"liver",
    "Stomach":"stomach",
    "Lungs": ['lung_upper_lobe_left', 'lung_lower_lobe_left','lung_upper_lobe_right', 'lung_middle_lobe_right','lung_lower_lobe_right'],
    'Colon':"colon",
    'Subcutaneous fat':"subcutaneous_fat",
    "Skeletal muscle":"skeletal_muscle",
    'Visceral fat':"visceral_fat",
    "Gray matter":['Left-Cerebral-Cortex', 'Right-Cerebral-Cortex'], 
    "White matter": ['Right-Cerebral-White-Matter' , 'Left-Cerebral-White-Matter'],
    "Brain":"brain",
    "Esophagus": "esophagus",
    "Trachea": "trachea",
    "Small intestine": ["small_bowel","duodenum"],
    "Skull": "skull",
    "Ribs": [f"rib_left_{i}" for i in range(1,13)] + [f"rib_right_{i}" for i in range(1,13)],
    "Urinary bladder": "urinary_bladder",
    "Gallbladder": "gallbladder",
    "Pancreas": "pancreas",
    "Heart": "heart",
    "Prostate": "prostate",
    "Thyroid": "thyroid_gland",
    "Spinal Cord": "spinal_cord",
    "Hip": ["hip_right","hip_left"],
    "Femur": ["femur_left","femur_right"],
    "Humerus": ["humerus_left","humerus_right"],
    "Vena cava": ["superior_vena_cava","inferior_vena_cava"],
    "Clavicula": ["clavicula_left","clavicula_right"],
    "Scapula": ["scapula_left","scapula_right"],
    "Vertebral column": ["sacrum","vertebrae_S1"] + [f"vertebrae_L{i}" for i in range(1,6)]+ [f"vertebrae_T{i}" for i in range(1,13)]+ [f"vertebrae_C{i}" for i in range(1,8)]
}






In [31]:
df = pd.read_pickle("/homes/hinge/Projects/hedyPET/readouts/means_80.pkl.gz")
df1 = filter_and_combine_regions(df,regions)
df1 = df1[["Subject", "Erosion Iterations","mu","region","Volume [mL]"]]
df1.to_pickle("means.pkl.gz")
df1

/tmp/ipykernel_2756449/1145753329.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["mu"] = result["PET Mean [Bq/mL]"]
/tmp/ipykernel_2756449/1145753329.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Volume [mL]"] = result["Volume [mL]"]
/tmp/ipykernel_2756449/1145753329.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,Subject,Erosion Iterations,mu,region,Volume [mL]
0,sub-072,0,657.080078,Spleen,199.276110
1,sub-072,1,670.445862,Spleen,167.836680
2,sub-063,0,714.929688,Spleen,126.846720
3,sub-063,1,731.791504,Spleen,102.246210
4,sub-025,0,1041.177002,Spleen,215.943255
...,...,...,...,...,...
5211,sub-095,1,742.548120,Vertebral column,732.303495
5212,sub-096,0,499.928658,Vertebral column,1144.332090
5213,sub-096,1,528.202580,Vertebral column,713.322225
5214,sub-098,0,404.036683,Vertebral column,1048.832235


In [58]:

df = pd.read_pickle("/homes/hinge/Projects/hedyPET/readouts/tacs_80.pkl.gz")
df = filter_and_combine_regions(df,regions)
t = [1,3,5,7,9,11.0,13.0,15.0,17.0,19.0,21.0,23.0,25.0,27.0,29.0,31.0,33.0,35.0,37.0,39.0,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5,95.0,105,115,125,135,145,155,165,175,185,195,205,215,225,235,270,330,390,450,510,570,660,780,900,1020,1140,1260,1380,1500,1620,1740,1950,2250,2550,2850,3150,3450,3750,4050]
df["t"] = df["Frame Index"].map(lambda x: t[int(x)])
df = df[["Subject", "Erosion Iterations","mu","region","Volume [mL]","Frame Index", "t"]]
df.to_pickle("tacs.pkl.gz")

/tmp/ipykernel_2756449/1145753329.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["mu"] = result["PET Mean [Bq/mL]"]
/tmp/ipykernel_2756449/1145753329.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Volume [mL]"] = result["Volume [mL]"]
/tmp/ipykernel_2756449/1145753329.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [59]:
df

,Subject,Erosion Iterations,mu,region,Volume [mL],Frame Index,t
0,sub-072,0,0.000000,Spleen,199.365000,0,1.0
1,sub-072,0,0.000000,Spleen,199.365000,1,3.0
2,sub-072,0,0.000000,Spleen,199.365000,2,5.0
3,sub-072,0,0.000000,Spleen,199.365000,3,7.0
4,sub-072,0,0.000000,Spleen,199.365000,4,9.0
...,...,...,...,...,...,...,...
359899,sub-098,1,434.999244,Vertebral column,649.336669,64,2850.0
359900,sub-098,1,433.636755,Vertebral column,649.336669,65,3150.0
359901,sub-098,1,432.783852,Vertebral column,649.336669,66,3450.0
359902,sub-098,1,429.925289,Vertebral column,649.336669,67,3750.0


In [37]:
def filter_and_combine_regions_ki(df,regions):
    group_cols = ["Subject","Erosion Iterations","Task","Input Function Identifier","Regression Frames"]
    results = []

    for k, v in regions.items():
        mask = df["Label Name"].isin(v) if isinstance(v, list) else df["Label Name"] == v
        filtered = df[mask]
        
        if isinstance(v, list):
            result = filtered.groupby(group_cols).apply(
                lambda x: pd.Series({
                    'slope': (x["Patlak Ki"] * x["Voxel Count"]).sum() / x["Voxel Count"].sum(),
                    'Volume [mL]': x['Volume [mL]'].sum()
                })
            ).reset_index()
        else:
            result = filtered
            result["slope"] = result["Patlak Ki"]
            #result["Volume [mL]"] = result["Volume [mL]"]
        result['region'] = k
        results.append(result)
    df = pd.concat(results, ignore_index=True)
    return df

In [49]:
df1 = pd.read_pickle("/homes/hinge/Projects/hedyPET/readouts/patlak_ki_80.pkl.gz")
df2 = df1.drop(["Label Index"],axis=1)
df2 = filter_and_combine_regions_ki(df2,regions)
df2["aorta_voi"] = df2["Input Function Identifier"].apply(lambda x: " ".join(str(x).split("_")[4:]))
df2["voi_volume"] = df2["Input Function Identifier"].apply(lambda x: "_".join(x.split("_")[1:3])).map({'ml-1_width-3':"1mL, 3px width", 'ml-1.5_width-5':"1.5mL, 5px width", 'ml-2_width-5':"2mL, 5px width"})
df2 = df2.drop(["Input Function Identifier"],axis=1)
df2 = df2[["Subject","Erosion Iterations","Regression Frames","Volume [mL]","slope","region","aorta_voi","voi_volume"]]
df2.to_pickle("patlak.pkl.gz")

/tmp/ipykernel_2756449/3201644927.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["slope"] = result["Patlak Ki"]
/tmp/ipykernel_2756449/3201644927.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['region'] = k
/tmp/ipykernel_2756449/3201644927.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude

In [56]:
import pandas as pd
df1 = pd.read_csv("/homes/hinge/Projects/hedyPET/readouts/metadata_80.csv")
df1["demographic-group"] = df1["Demographic Group [Sex,Age]"].str[1:]
df1["sex"] = df1["Demographic Group [Sex,Age]"].str[:1]
df1.to_pickle("norm2.pkl")